In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.types import *

In [3]:
from pyspark.sql.functions import *

In [4]:
spark=SparkSession.builder.getOrCreate()

In [5]:
ds1=spark.read.csv('202105-divvy-tripdata.csv', header=True)
ds2=spark.read.csv('202104-divvy-tripdata.csv', header=True)
ds3=spark.read.csv('202103-divvy-tripdata.csv', header=True)
ds4=spark.read.csv('202102-divvy-tripdata.csv', header=True)
ds5=spark.read.csv('202101-divvy-tripdata.csv', header=True)
ds6=spark.read.csv('202012-divvy-tripdata.csv', header=True)
ds7=spark.read.csv('202011-divvy-tripdata.csv', header=True)
ds8=spark.read.csv('202010-divvy-tripdata.csv', header=True)
ds9=spark.read.csv('202009-divvy-tripdata.csv', header=True)
ds10=spark.read.csv('202008-divvy-tripdata.csv', header=True)
ds11=spark.read.csv('202007-divvy-tripdata.csv', header=True)
ds12=spark.read.csv('202006-divvy-tripdata.csv', header=True)

In [7]:
#merging all datasets to one 'ds' dataframe
ds=ds1.union(ds2).union(ds3).union(ds4).union(ds5).union(ds6).union(ds7).union(ds8).union(ds9).union(ds10).union(ds11).union(ds12)

In [9]:
print((ds.count(), len(ds.columns)))

(4073561, 13)


In [11]:
ds.first()

Row(ride_id='C809ED75D6160B2A', rideable_type='electric_bike', started_at='2021-05-30 11:58:15', ended_at='2021-05-30 12:10:39', start_station_name=None, start_station_id=None, end_station_name=None, end_station_id=None, start_lat='41.9', start_lng='-87.63', end_lat='41.89', end_lng='-87.61', member_casual='casual')

In [14]:
ds.show(5)

+----------------+-------------+-------------------+-------------------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|start_station_name|start_station_id|end_station_name|end_station_id|start_lat|start_lng|end_lat|end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+------------------+----------------+----------------+--------------+---------+---------+-------+-------+-------------+
|C809ED75D6160B2A|electric_bike|2021-05-30 11:58:15|2021-05-30 12:10:39|              null|            null|            null|          null|     41.9|   -87.63|  41.89| -87.61|       casual|
|DD59FDCE0ACACAF3|electric_bike|2021-05-30 11:29:14|2021-05-30 12:14:09|              null|            null|            null|          null|    41.88|   -87.62|  41.79| -87.58|       casual|
|0AB83CB88C43EFC2|electric_bike|2021-05-30 14

In [15]:
ds.dropDuplicates().count()

4073561

In [16]:
import pyspark.sql.functions as F
ds = ds.withColumn("a", (
        F.pow(F.sin(F.radians(F.col("end_lat") - F.col("start_lat")) / 2), 2) +
        F.cos(F.radians(F.col("start_lat"))) * F.cos(F.radians(F.col("end_lat"))) *
        F.pow(F.sin(F.radians(F.col("end_lng") - F.col("start_lng")) / 2), 2)
    )).withColumn("distance_traveled", F.atan2(F.sqrt(F.col("a")), F.sqrt(-F.col("a") + 1)) * 12742000)

In [21]:
ds.select('start_lat','start_lng', 'end_lat', 'end_lng','distance_traveled').show(5)

+---------+---------+-------+-------+------------------+
|start_lat|start_lng|end_lat|end_lng| distance_traveled|
+---------+---------+-------+-------+------------------+
|     41.9|   -87.63|  41.89| -87.61|1994.1890794035653|
|    41.88|   -87.62|  41.79| -87.58|10541.961176829122|
|    41.92|    -87.7|  41.92|  -87.7|               0.0|
|    41.92|    -87.7|  41.94| -87.69| 2372.775481761235|
|    41.94|   -87.69|  41.94|  -87.7| 827.1180459695228|
+---------+---------+-------+-------+------------------+
only showing top 5 rows



In [23]:
#create a column that finds the date difference, and show it in descending order (largest to least)
ds= ds.withColumn('date_diff', F.datediff(F.to_date(ds.ended_at), F.to_date(ds.started_at)))
ds.select('started_at', 'ended_at', 'member_casual','date_diff').sort(ds.date_diff.desc()).show(10)

+-------------------+-------------------+-------------+---------+
|         started_at|           ended_at|member_casual|date_diff|
+-------------------+-------------------+-------------+---------+
|2020-09-02 18:34:33|2020-10-10 11:17:54|       casual|       38|
|2021-05-02 02:56:07|2021-06-08 13:37:43|       casual|       37|
|2020-09-06 23:20:29|2020-10-12 11:46:25|       casual|       36|
|2020-09-05 08:50:15|2020-10-10 13:43:02|       casual|       35|
|2020-07-05 14:25:39|2020-08-09 07:11:06|       casual|       35|
|2020-07-05 01:51:06|2020-08-08 12:13:19|       casual|       34|
|2020-07-02 17:26:55|2020-08-04 07:16:12|       casual|       33|
|2021-04-02 17:50:00|2021-05-05 22:06:42|       casual|       33|
|2020-07-02 19:49:10|2020-08-04 18:00:37|       casual|       33|
|2020-07-07 14:36:11|2020-08-09 19:13:11|       casual|       33|
+-------------------+-------------------+-------------+---------+
only showing top 10 rows



In [26]:
#when we order the date difference in ascending order, we see that we have incorrect negative values 
ds.select('started_at', 'ended_at', 'member_casual','date_diff').sort(ds.date_diff.asc()).show(10)

+-------------------+-------------------+-------------+---------+
|         started_at|           ended_at|member_casual|date_diff|
+-------------------+-------------------+-------------+---------+
|2021-05-28 12:48:58|2021-05-28 13:00:08|       casual|        0|
|2021-05-30 19:45:10|2021-05-30 20:01:31|       casual|        0|
|2021-05-26 23:42:44|2021-05-26 23:53:55|       casual|        0|
|2021-05-13 22:43:48|2021-05-13 22:50:27|       member|        0|
|2021-05-13 22:43:51|2021-05-13 22:50:33|       member|        0|
|2021-05-05 08:18:53|2021-05-05 08:38:34|       member|        0|
|2021-05-17 12:29:42|2021-05-17 12:57:30|       member|        0|
|2021-05-13 07:47:26|2021-05-13 12:29:02|       member|        0|
|2021-05-01 16:38:28|2021-05-01 16:48:51|       casual|        0|
|2021-05-08 05:12:06|2021-05-08 05:20:39|       casual|        0|
+-------------------+-------------------+-------------+---------+
only showing top 10 rows



In [25]:
ds=ds.filter(col('date_diff').cast(LongType()) >= 0)
ds.count()

4073182

In [27]:
ds = ds.withColumn('duration_in_min', (ds.date_diff*24*60) + F.hour(ds.ended_at)*60+F.minute(ds.ended_at)+
        F.second(ds.ended_at)/60- F.hour(ds.started_at)*60-F.minute(ds.started_at)- F.second(ds.started_at)/60)

In [28]:
ds.select('started_at', 'ended_at', 'member_casual','date_diff', 'distance_traveled', 'duration_in_min').show(10)

+-------------------+-------------------+-------------+---------+------------------+------------------+
|         started_at|           ended_at|member_casual|date_diff| distance_traveled|   duration_in_min|
+-------------------+-------------------+-------------+---------+------------------+------------------+
|2021-05-30 11:58:15|2021-05-30 12:10:39|       casual|        0|1994.1890794035653|12.399999999999977|
|2021-05-30 11:29:14|2021-05-30 12:14:09|       casual|        0|10541.961176829122| 44.91666666666664|
|2021-05-30 14:24:01|2021-05-30 14:25:13|       casual|        0|               0.0|1.2000000000000304|
|2021-05-30 14:25:51|2021-05-30 14:41:04|       casual|        0| 2372.775481761235| 15.21666666666672|
|2021-05-30 18:15:39|2021-05-30 18:22:32|       casual|        0| 827.1180459695228| 6.883333333333303|
|2021-05-30 11:33:41|2021-05-30 11:57:17|       casual|        0|1386.2667237514747| 23.59999999999997|
|2021-05-30 10:51:37|2021-05-30 11:06:20|       casual|        0

In [30]:
ds.select('distance_traveled','duration_in_min').summary().show()

+-------+------------------+-------------------+
|summary| distance_traveled|    duration_in_min|
+-------+------------------+-------------------+
|  count|           4068146|            4073182|
|   mean|   2221.8403058682| 26.882429780942815|
| stddev|2025.9532818899693|  236.6782296468922|
|    min|               0.0|-120.30000000000004|
|    25%| 865.4421767937478|  7.666666666666621|
|    50%|1675.2243117725307| 14.016666666666605|
|    75%|3018.4568996381654| 25.850000000000016|
|    max| 48370.80097108494|           54283.35|
+-------+------------------+-------------------+



In [31]:
ds.select('started_at', 'ended_at', 'date_diff', 'duration_in_min').sort(ds.duration_in_min.asc()).show(10)


+-------------------+-------------------+---------+-------------------+
|         started_at|           ended_at|date_diff|    duration_in_min|
+-------------------+-------------------+---------+-------------------+
|2020-07-25 15:08:21|2020-07-25 13:08:03|        0|-120.30000000000004|
|2020-10-16 16:44:52|2020-10-16 15:09:51|        0| -95.01666666666664|
|2020-10-16 16:44:53|2020-10-16 15:10:54|        0| -93.98333333333336|
|2020-10-16 16:44:55|2020-10-16 15:11:27|        0| -93.46666666666663|
|2020-10-16 16:44:56|2020-10-16 15:43:14|        0| -61.69999999999998|
|2020-10-16 16:44:58|2020-10-16 15:45:03|        0|-59.916666666666714|
|2020-11-01 01:57:30|2020-11-01 01:00:39|        0|             -56.85|
|2020-11-01 01:56:26|2020-11-01 01:00:29|        0|-55.949999999999996|
|2020-11-01 01:55:57|2020-11-01 01:02:04|        0| -53.88333333333333|
|2020-11-01 01:54:40|2020-11-01 01:01:34|        0|-53.099999999999994|
+-------------------+-------------------+---------+-------------

In [32]:
ds=ds.filter(col('duration_in_min') >= 0.0)
ds.count()

4063030

In [33]:
ds=ds.withColumn("day_of_week", date_format(col('started_at'), 'EEEE'))

In [35]:
ds.select('member_casual','distance_traveled', 'duration_in_min', 'day_of_week').summary().show()

+-------+-------------+------------------+------------------+-----------+
|summary|member_casual| distance_traveled|   duration_in_min|day_of_week|
+-------+-------------+------------------+------------------+-----------+
|  count|      4063030|           4058030|           4063030|    4063030|
|   mean|         null|2221.7196330920974|26.951445087369166|       null|
| stddev|         null|2025.9456418320642|236.96962416906916|       null|
|    min|       casual|               0.0|               0.0|     Friday|
|    25%|         null| 865.4421767937478| 7.700000000000012|       null|
|    50%|         null|1674.9411822309112|14.049999999999985|       null|
|    75%|         null|3018.4110032739354|25.899999999999984|       null|
|    max|       member| 48370.80097108494|          54283.35|  Wednesday|
+-------+-------------+------------------+------------------+-----------+



In [36]:
#casual vs member distribution
ds.cube('member_casual').count().show()

+-------------+-------+
|member_casual|  count|
+-------------+-------+
|       casual|1710107|
|         null|4063030|
|       member|2352923|
+-------------+-------+



In [37]:
#frequency distribution of day of the week
ds.cube('day_of_week').count().show()

+-----------+-------+
|day_of_week|  count|
+-----------+-------+
|  Wednesday| 529720|
|   Thursday| 533708|
|    Tuesday| 503792|
|     Monday| 503884|
|       null|4063030|
|     Friday| 597210|
|     Sunday| 637741|
|   Saturday| 756975|
+-----------+-------+



In [38]:
#frequency distr of bike types
ds.cube('rideable_type').count().show()

+-------------+-------+
|rideable_type|  count|
+-------------+-------+
| classic_bike| 843555|
|         null|4063030|
|electric_bike| 888224|
|  docked_bike|2331251|
+-------------+-------+



In [39]:
#start_station_name frequency is counted and sorted in a descending way
df=spark.createDataFrame(ds.cube('start_station_name').count().collect())


In [41]:
df.sort(df['count'].desc()).show(10)

+--------------------+-------+
|  start_station_name|  count|
+--------------------+-------+
|                null|4063030|
|                null| 201925|
|Streeter Dr & Gra...|  48000|
|Lake Shore Dr & M...|  39335|
|   Clark St & Elm St|  35982|
| Theater on the Lake|  35935|
|Lake Shore Dr & N...|  32401|
|     Millennium Park|  30385|
|Wells St & Concor...|  29948|
|Michigan Ave & Oa...|  28524|
+--------------------+-------+
only showing top 10 rows



In [42]:
#filling all null values with missing_data value, so that we can infer smth in EDA
ds=ds.na.fill('missing_data')

In [43]:
ds=ds.orderBy('started_at')

In [46]:
print((ds.count(), len(ds.columns)))

(4063030, 18)


In [47]:
ds=ds.drop('ride_id')
ds=ds.drop('start_station_id')
ds=ds.drop('end_station_id')
ds=ds.drop('start_lat')
ds=ds.drop('end_lat')
ds=ds.drop('start_lng')
ds=ds.drop('end_lng')
ds=ds.drop('a')
ds=ds.drop('date_diff')

In [18]:
ds.repartition(1).write.csv("ds_dropped.csv", header=True)